# Integration for Instagram in Python

This script was created to automate some activities inherent to page maintenance on Instagram. 
Among these activities are unfollowing a follower without interaction and sending a welcome message to new followers.
For this, followers, followings, posts, likes and comments are extracted from Instagram and stored in an SQLite database.

# Imports and setups

In [ ]:
# General Importing
import pandas as pd
from pandas.io.json import json_normalize 
import os, time, sys, csv, glob
from datetime import datetime, timedelta, timezone
from random import randint
from IPython.core.interactiveshell import InteractiveShell

# Importing instagrapi
from instagrapi import Client 

# Importing libraries for database
import sqlite3 #https://appdividend.com/2020/10/14/how-to-create-sqlite-database-in-python/
import sqlalchemy as db #https://appdividend.com/2020/10/15/how-to-use-sqlalchemy-core-with-sqlite-in-python/

# Importing own functions
import functions_insta as fi
import functios_sqlite as lite

# Basic setup in Jupyter 
InteractiveShell.ast_node_interactivity = "all"

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Basic setup for graphics and other output
pd.options.display.max_columns = 500

# Creating required tables in SQLite

Run this cell only when you want to recreate the tables. THE TABLES WILL BE DROPPED ! ! !

In [ ]:
# lite.cria_tbl_followers()

# lite.cria_tbl_followings()

# lite.cria_tbl_posts()

# lite.cria_tbl_likes()

# lite.cria_tbl_comments()

# lite.cria_tbl_convites_instagrow()

# Defining variables

In [ ]:
conta = '12345678901' # Instagram account
usr='your_user' # Instagram User
psw='your_password' # Instagram Password

# List of my users who follow Instagram profile
nossos_usernames = ['profile1', '...' , 'profilen'] 

# List of users that I will not unfollow
nao_excluir = ['profile1', '...', 'profilen'] 

# Defining time periods for analysis and locations to save files
path_raiz = os.getcwd()
print(path_raiz) 
data_hora_atual = str(datetime.today()).replace(":","").replace(".","")[:17]
print(data_hora_atual)
dias_para_extrair = 115
data_de_corte = datetime.today() - timedelta(days=dias_para_extrair) 
print('data_de_corte:', data_de_corte) 

dt_temp = time.localtime() 
dt_hoje = str(dt_temp.tm_year) + str(dt_temp.tm_mon).zfill(2) + str(dt_temp.tm_mday).zfill(2)
print('dt_hoje:', dt_hoje) 

# Log in to Instagram

Log in prepared for MFA.

In [ ]:
fi.aux_login_instagrapi(usr, psw) #https://adw0rd.github.io/instagrapi/usage-guide/interactions.html

# Extracting followings

In [ ]:
df_followings = fi.extrai_followings_instagrapi(path_raiz, conta)

len(df_followings)
df_followings.head()

# Import followings from csv file

Uncomment and execute this cell only when you prefer to import a previously generated followings file instead of extracting it again from Instagram.

In [ ]:
'''
df_followings = pd.read_csv(path_raiz + 'your_historic_file.csv', sep=';') 
df_followings.drop(['Unnamed: 0'], axis=1, inplace = True) 
df_followings.head()
'''

# Extracting followers

In [ ]:
# Extract followers and create dataframe
df_followers = fi.extrai_followers_instagrapi(path_raiz, conta)

len(df_followers)
df_followers.head()

data_atualizacao = dt_hoje[0:4] + '-'+ dt_hoje[4:6] + '-'+ dt_hoje[6:8]
print(data_atualizacao) 

# Update table followers in SQLite
lite.atualiza_followers(df_followers, data_atualizacao)

# Import followers from CSV file

Uncomment and execute this cell only when you prefer to import a previously generated followers files instead of extracting it again from Instagram.

In [ ]:
'''
path = r'your_folder\CSVs'

# Fetch all previously extracted followers files in the folder
os.chdir(path)

arquivos = []

for arquivo in os.listdir():
    if 'df_followers' in arquivo:
        arquivos.append(arquivo)
arquivos.sort()

# Reads each file in the list and inserts its data into the followers table in SQLite
os.chdir(r'your_folder')

for arquivo in arquivos:
    data_atualizacao = arquivo[13:17] + '-'+arquivo[18:20] + '-'+arquivo[21:23]

    print(data_atualizacao, ' - ', arquivo) 
    
    df_followers = pd.read_csv(path_raiz + '\\CSVs\\' + arquivo, sep=';')

    lite.atualiza_followers(df_followers, data_atualizacao)
'''

# Send welcome message

### Create list of new followers.

In [ ]:
boas_vindas = {}

conn = sqlite3.connect('instagram.db')
cur = conn.cursor()
 
# https://www.sqlite.org/lang_select.html
sqlstr = """select count(1)
            from followers f 
            where f.welcome is null"""
    
# https://www.sqlite.org/lang_select.html
sqlstr = """select 
                f.username, 
                CASE when f.full_name is null then f.username else f.full_name end as convites 
            from followers f 
            where f.welcome is null and data_saida is null
            """

for row in cur.execute(sqlstr):
    boas_vindas[str(row[0])] = row[1]

cur.close()

print('List generated at', datetime.today())
boas_vindas

### Send messages

In [ ]:
from instabot import Bot

print('Starting sending welcome at', datetime.today()) 
tempo_de_espera = 1
qtde = len(boas_vindas)
count = 0

# Message
text = 'Hello, {}!\nWelcome to our page.\nWe prepare our posts with great care.\nI love receiving comments on posts and look forward to yours. Send it there, okay?\nGratitude 🌹💖🙌'

# Creating bot varible.
bot = Bot()

# Login using bot.
print('Logging in')
bot.login(username=usr, password=psw)

for k, v in boas_vindas.items(): 
    count += 1 
    
    # Program to send message
    # on Instagram using Python.
    # para caso dê erro de JSON. Solução de Adarsh chaurasia
    # https://stackoverflow.com/questions/66794193/cant-login-with-instabot
    # https://github.com/ohld/igbot/blob/d68b55015322898c3842c92dcc5989581043252e/examples/welcome_message.py
    # https://codingshiksha.com/python/python-3-instagram-api-script-to-send-direct-messages-to-multiple-usernames-using-instabot-module-full-project-for-beginners/

    # Sending messages
    status = bot.send_message(text.format(v), k)
    print(count, 'of', qtde, 'user:', k,  '- status:', status, 'sent at', datetime.today())

    # Update status to welcomed
    if status == False: break 
    else: 
        update_registro = "UPDATE new_followers SET welcome = 1 where username = '%s'"

        conn = sqlite3.connect('instagram.db')
        cur = conn.cursor()
        cur.execute(update_registro %(k)) 
        conn.commit()
        conn.close() 
    
print('Welcome submission completed at', datetime.today()) 

# Extracting posts

In [ ]:
df_my_posts = fi.extrai_posts_instagrapi(path_raiz, conta)
len(df_my_posts)

# Inserting posts at posts table
lite.insere_posts(df_my_posts)

# Import posts from CSV file

Uncomment and execute this cell only when you prefer to import a previously generated posts file instead of extracting it again from Instagram.

In [ ]:
'''
df_my_posts = pd.read_csv(path_raiz + '\CSVs\df_my_posts_2022-04-25 180849.csv', sep=';') 
df_my_posts.drop(['Unnamed: 0'], axis=1, inplace = True) 
df_my_posts['criado_em_post'] = pd.to_datetime(df_my_posts['criado_em_post']) 
#df_my_posts.info()
df_my_posts.head()
'''

# Export posts

In [ ]:
# Filtering only posts created after a cutoff date
filtro1_posts = df_my_posts['criado_em_post'] >= data_de_corte 

# Extract posts
my_posts_temp = list(df_my_posts[filtro1_posts]['post_id'])
len(my_posts_temp)
my_posts_temp[0:2]

# Export comments

In [ ]:
df_my_comments = fi.extrai_comentarios_instagrapi(path_raiz, my_posts_temp)
len(df_my_comments)
df_my_comments.head()
#lite.insere_posts(df_my_comments)

# Import comments from CSV file

Uncomment and execute this cell only when you prefer to import a previously generated comments file instead of extracting it again from Instagram.

In [ ]:
'''
df_my_comments = pd.read_csv(path_raiz + 'your_comments_file.csv', sep=';') 
df_my_comments.drop(['Unnamed: 0'], axis=1, inplace = True) 
df_my_comments.head()
'''

# Extract likes

In [ ]:
df_my_likes = fi.extrai_curtidas_instagrapi(path_raiz, my_posts_temp)
len(df_my_likes)
df_my_likes.head()

# Import likes from CSV file

Uncomment and execute this cell only when you prefer to import a previously generated likes file instead of extracting it again from Instagram.

In [ ]:
''' 
df_my_likes = pd.read_csv(path_raiz + 'your_likes_file.csv', sep=';') 
df_my_likes.drop(['Unnamed: 0'], axis=1, inplace = True) 
df_my_likes.head() 
'''

# Finds followed without interaction with the page

In [ ]:
df_followings['pk'] = pd.to_numeric(df_followings['pk'])
df_my_likes = df_my_likes.rename(columns={'follower_pk':'pk'})
df_my_comments = df_my_comments.rename(columns={'list_follower_pk':'pk'})

df_followings_s_like = df_followings[~df_followings['pk'].isin(df_my_likes['pk'])]
df_followings_s_like_s_comment = df_followings_s_like[~df_followings_s_like['pk'].isin(df_my_comments['pk'])]

data_hora_atual = str(datetime.today()).replace(":","").replace(".","")[:17]    
df_followings_s_like_s_comment.to_csv(path_raiz + '\\CSVs\\df_followings_s_like_s_comment_' + data_hora_atual + '.csv', encoding='utf-8', sep=';', decimal=',') 

# Unfollow user

Some of the techniques used are to avoid Instagram blocks. Instagram blocks the page for a while and requires a new login when there are many unfollows in a short period of time, for example.

In [ ]:
excluir = ['12345678901'
    ]

print('Starting unfollow at', datetime.today()) 
tempo_de_espera = 65 #(with 85 it works fine)
descanso = 15 # in minutes
count = 0 
de = 0
ate = len(excluir)

for i in excluir:
    count += 1 
    qtde = ate - de
    
    try:
        status = fi.deixa_de_seguir(i) 
    except: # AssertionError as error:
        status = None 

    print(count, 'de', qtde, '- id:', i,  '- status:', status)
    if status == False: break 
        
    if count == qtde: print('Até a próxima pe pe pe pe pessoal') 
    elif count % 50 == 0:
        print("It's", datetime.today(), ". Let's wait", descanso*4, 'minutes.\n')
        time.sleep(descanso * 4 * 60)
    elif count % 17 == 0:
        print("It's", datetime.today(), ". Let's wait", descanso, 'minutes.\n')
        time.sleep(descanso * 60) 
    elif count != qtde: 
        espera = tempo_de_espera * (1 + randint(-50,30)/100) 
        print('Waiting', round(espera, 2), 'seconds')
        time.sleep(espera)


print('Unfollow completed at', datetime.today()) 